In [28]:
import module.TorchDataset as td
import platform
import importlib

importlib.reload(td) # 클래스 수정 시 import 리로드

# os에 따라 다르게 적용 (코랩은 리눅스)
current_os = platform.system()
sep = "\\" if current_os == "Windows" else "/" 


# TrainSet
root_train = f"..{sep}01.Data{sep}fire{sep}train{sep}" # 루트폴더
extension = ["jpg", "png"] # 찾을 확장자
img_size = 224

train_dataset = td.TorchDataset(root_train, sep, extension, img_size, "train", False)


# ValidSet
root_valid = f"..{sep}01.Data{sep}fire{sep}valid{sep}" # 루트폴더
valid_dataset = td.TorchDataset(root_valid, sep, extension, img_size, "test", False)


# TestSet
root_test = f"..{sep}01.Data{sep}fire{sep}test{sep}" # 루트폴더
test_dataset = td.TorchDataset(root_test, sep, extension, img_size, "test", False)

len(train_dataset), len(valid_dataset), len(test_dataset)

(43982, 6300, 6299)

In [4]:
# print(train_dataset.get_label_dict())
# train_dataset.get_sample_image(8, True)
# valid_dataset.get_sample_image(8, True)
# test_dataset.get_sample_image(8, True)

In [5]:
# train_dataset.get_target_ratio("Train")
# valid_dataset.get_target_ratio("Valid")
# test_dataset.get_target_ratio("Test")

In [29]:
import module.TorchDatasetCNN as tdCNN
importlib.reload(tdCNN)

cnn_runner = tdCNN.TorchDatasetCNN()
batch_size = 20

train_dataloader = cnn_runner.get_dataloader(train_dataset, batch_size, True, True)
valid_dataloader = cnn_runner.get_dataloader(valid_dataset, batch_size, False, False)
test_dataloader = cnn_runner.get_dataloader(test_dataset, batch_size, True, False)
len(train_dataloader), len(valid_dataloader), len(test_dataloader)

(2199, 315, 315)

In [7]:
######################### ResNET50 ########################
###########################################################
###########################################################
###########################################################

In [ ]:
import torch.nn as nn
from torchvision import models
from torchsummary import summary

epoch = 10
device = cnn_runner.get_device()
learning_rate = 0.001
save_path = ""

save_path = ""
if current_os == "Windows": # 로컬 저장용
    save_path = f"save_model{sep}03.Torch_CNN{sep}03.resnet50_Header_Finetune{sep}resnet50_Header_Finetune.pth"
else:
    save_path = "/content/resnet50_Header_Finetune.pth" # 코랩 저장용

model = models.resnet50(weights='ResNet50_Weights.DEFAULT').to(device)

# 모든 파라미터 Freezing
for param in model.parameters():
    param.requires_grad = False
    
# 마지막 레이어 layer4 파라미터만 풀어주기
for param in model.layer4[2].parameters():
    param.requires_grad = True


# fc layer만 수정
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

# trainable parameter 수 확인
summary(model, input_size = (3, 224, 224))

In [ ]:
print(model)

In [ ]:
# Train
train_acc, valid_acc, valid_recall = cnn_runner.run_epoch(epoch, train_dataloader, valid_dataloader, device, model, learning_rate, save_path)

In [ ]:
# 모델 로딩 및 테스트
save_path = f"E:{sep}02.공부{sep}02.코딩{sep}01.Python{sep}01.Alphaco{sep}02.코드{sep}03.프로젝트{sep}03.이미지분류{sep}00.Code{sep}save_model{sep}03.Torch_CNN{sep}05.resnet50_Layer4_Finetune{sep}resnet50_Layer4_Finetune.pth"
model = cnn_runner.load_model(model, save_path, device)
# cnn_runner.test_model(test_dataloader, device, model)